In [ ]:
# Handle missing values (fill with empty strings)
'''df['combined_text'] = df['combined_text'].fillna('')
print(df.head())'''

         id                                title   type  \
0  ts300399  Five Came Back: The Reference Films   SHOW   
1   tm82169                                Rocky  MOVIE   
2   tm17823                               Grease  MOVIE   
3  tm191099                            The Sting  MOVIE   
4   tm69975                             Rocky II  MOVIE   

                                         description  release_year  \
0  This collection includes 12 World War II-era p...          1945   
1  When world heavyweight boxing champion, Apollo...          1976   
2  Australian good girl Sandy and greaser Danny f...          1978   
3  A novice con man teams up with an acknowledged...          1973   
4  After Rocky goes the distance with champ Apoll...          1979   

  age_certification  runtime                                 genres  \
0             TV-MA       51                      ['documentation']   
1                PG      119                     ['drama', 'sport']   
2          

In [1]:
!pip install pandas sentence-transformers chromadb streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [2]:
import pandas as pd

# Load the dataset from the URL
url = "https://raw.githubusercontent.com/datum-oracle/netflix-movie-titles/main/titles.csv"
df = pd.read_csv(url)

# Combine relevant columns for text embeddings
df['combined_text'] = df['title'] + " " + df['description'] + " " + df['genres']

In [3]:
!pip install chromadb --upgrade

In [4]:
from sentence_transformers import SentenceTransformer
import chromadb

# Load the pre-trained model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for the combined text
embeddings = model.encode(df['combined_text'].tolist())

# Initialize ChromaDB and manage the collection
client = chromadb.Client()

# Check and delete existing collection if needed
if "movies" in [col.name for col in client.list_collections()]:
    client.delete_collection("movies")

# Create a fresh collection
collection = client.create_collection("movies")

# Generate unique IDs for each document
ids = [str(i) for i in range(len(df))]

# Index embeddings in ChromaDB
collection.add(
    ids=ids,  # List of unique IDs
    documents=df['combined_text'].tolist(),  # The text data to index
    embeddings=embeddings,  # The corresponding embeddings
    metadatas=[{"title": title} for title in df['title']]  # Metadata for each entry
)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
!pip install pyngrok
from pyngrok import ngrok

In [6]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import chromadb

# Load data and model
url = "https://raw.githubusercontent.com/datum-oracle/netflix-movie-titles/main/titles.csv"
df = pd.read_csv(url)
df['combined_text'] = df['title'] + " " + df['description'] + " " + df['genres']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['combined_text'].tolist())

# Initialize ChromaDB client
client = chromadb.Client()

# Check if the collection exists
existing_collections = [col.name for col in client.list_collections()]
print("Existing collections:", existing_collections)

# Delete the collection if it exists
if "movies" in existing_collections:
    client.delete_collection("movies")
    print("Deleted existing collection 'movies'.")

# Create a fresh collection
collection = client.create_collection("movies")

# Generate unique IDs for each entry
ids = [str(i) for i in range(len(df))]

# Index the data in ChromaDB
collection.add(
    ids=ids,  # List of unique IDs
    documents=df['combined_text'].tolist(),  # The combined text data
    embeddings=embeddings,  # The generated embeddings
    metadatas=[{"title": df['title'][i]} for i in range(len(df))]  # Metadata for each entry
)

# Streamlit app
st.title("Movie Recommendation App")
user_input = st.text_input("Enter a movie description (e.g., heartfelt romantic comedy):")

if user_input:
    query_embedding = model.encode([user_input])
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    top_indices = np.argsort(similarities)[::-1][:5]

    st.write("Recommended Movies:")
    for idx in top_indices:
        st.write(df['title'][idx])


Writing app.py


In [7]:
!ngrok config add-authtoken 2kmYv8tzvcb6Xa97vBKhdVUxE4z_THBo5dnCi9svXquV8gSC


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok

# Use "http" instead of specifying a port number directly
public_url = ngrok.connect(8501)  # Correctly specify the port number without "port="

print(f"Streamlit app running at: {public_url}")



Streamlit app running at: NgrokTunnel: "https://5e68-34-148-12-217.ngrok-free.app" -> "http://localhost:8501"
